In [ ]:
import serial # import Serial Library 
import time
import cv2 #Import opencv
import EMPHUP_Lib_1

In [ ]:
arduinoData = serial.Serial('com3', 115200) #Connect to ESP32
mensaje = '2.7, 2.7, 0, 0, 1, 1, 0\n'
arduinoData.write(bytes(mensaje, 'utf-8')) #Servo, motor, LeftLed, RightLed, TOKYO, Autom, Vel fija
arduinoData.flush()

In [ ]:
current_time = 0.0
prev_time = 0.0
dt_ms = 80.0/1000.0

In [ ]:
#Connect to cel and configure settings
thres = 0.45
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('http://192.168.1.185:8080/videofeed')
cap.set(3,1280)
cap.set(4,720)
cap.set(10,70)

In [ ]:
#Import label names object detection
classNames = []
classFile = 'coco.names'
with open(classFile,'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')

In [ ]:
#Set configuration
configPath = "ssd_mobilenet_v3_large_coco_2020_01_14.pbtxt"
weightsPath = "frozen_inference_graph.pb"

#Load the neural network
net = cv2.dnn_DetectionModel(weightsPath,configPath)
net.setInputSize(320,320).setInputScale(1.0/ 127.5).setInputMean((127.5, 127.5, 127.5)).setInputSwapRB(True)

In [ ]:
#Variables de bt
i = 0
readString = ""
readList = []
iterator = 0
num_iter = 300
Motor = 2.7
ServoM = 2.7
Left = False
Right = False
Stop = False
Forward = True
prom_dist = 0.0

In [ ]:
current_time = time.time()
while True:
    left = []
    center = []
    right = []

    success,img = cap.read()
    cv2.line(img, (240,0), (240,480), color=(100,100,0), thickness=1)
    cv2.line(img, (480,0), (480,480), color=(100,100,0), thickness=1)
    classIds, confs, bbox = net.detect(img,confThreshold=thres)

    #Marcado de cajas y lineas en la imagen
    if len(classIds) != 0:
        for classId, confidence,box in zip(classIds.flatten(),confs.flatten(),bbox):
            cv2.rectangle(img,box,color=(0,255,0),thickness=1)
            cv2.putText(img,(classNames[classId-1].upper()+' '+str(round(confidence*100,2))),(box[0]+10,box[1]+30),
                        cv2.FONT_HERSHEY_COMPLEX,1,(255,100,0),2)
            if (box[0]<240):
                left.append(box[0])
            elif (240<=box[0]<480):
                center.append(box[0])
            else:
                right.append(box[0])
    if img is not None:
        cv2.imshow("Output",img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
                    
    #Bt data
    while(arduinoData.inWaiting()==0): #Wait here until there is data    
        pass #do nothing
    readString = arduinoData.readline().decode("utf-8")
    readList = readString.split(',')
    for num in range(6):
        readList[num] = float(readList[num])
    print(readList)
#     mensaje = '{}, {}, 0, 0, 0, 1, 0\n'.format(2.7, 2.7)
#     arduinoData.write(bytes(mensaje, 'utf-8'))
        
    prom_dist = (readList[1]+readList[2]+readList[3]) / 3
        
    if(((17 in classIds) or (16 in classIds) or (3 in classIds) or (0 in classIds) or (12 in classIds)) and (prom_dist <= 60)):
        Stop = True
    elif((1 in classIds) and (prom_dist <= 60)):
        Left = True
    elif((2 in classIds) and (prom_dist <= 60)):
        Right = True
    else:
        Forward = True
        
    if((not(17 in classIds) and not(16 in classIds) and not(3 in classIds) and not(0 in classIds) and not(12 in classIds))):
        Stop = False
    elif(not(1 in classIds)):
        Left = True
    elif(not(2 in classIds)):
        Right = True
        
    if(Stop == True) and (iterator < num_iter):
        ServoM, Motor, iterator = EMPHUP_Lib_1.Stop(iterator, num_iter)
        Left = False
        Right = False
        Forward = False
    elif(Left == True) and (iterator < num_iter):
        ServoM, Motor, iterator = EMPHUP_Lib_1.Izquierda(iterator, num_iter)
        Stop = False
        Right = False
        Forward = False
    elif(Right == True) and (iterator < num_iter):
        ServoM, Motor, iterator = EMPHUP_Lib_1.Derecha(iterator, num_iter)
        Left = False
        Stop = False
        Forward = False
    elif(iterator == num_iter):
        iterator = 0
    else:
        ServoM, Motor = EMPHUP_Lib_1.Derecho()
        Left = False
        Right = False
        Stop = False
        
    print(Stop, Left, Right, Forward, iterator)
    print(classIds)
    
    mensaje = '{}, {}, 0, 0, 0, 1, 0\n'.format(ServoM, Motor)
    arduinoData.write(bytes(mensaje, 'utf-8')) #Servo, motor, LeftLed, RightLed, TOKYO, Autom, Vel fija
    arduinoData.flush()
        
cap.release()

cv2.destroyAllWindows()

In [ ]:
arduinoData.close()